Install the OpenAI libray

In [75]:
%pip install openai  python-dotenv



Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 24.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


Load neccesary environment variables

In [76]:
# Import the OpenAI API key from the .env file
import os # Import the OS module
from dotenv import load_dotenv # Import the load_dotenv function from the dotenv module
load_dotenv() # Load the .env file

True

Get variables from the environment

In [77]:
# Get the OpenAI API key from the environment
openai_api_key = os.getenv('OPEN_AI_API_KEY') # Get the OpenAI API key from the environment



Create connection to OpenAI

In [78]:
from openai import OpenAI
client = OpenAI(api_key=openai_api_key) # Create a new OpenAI client

Initiate GPT assistant

In [79]:
assistant = client.beta.assistants.create(
    name="Kudu AI",
    instructions="You are a medical assistant and helps patients with questions around their medication. You can provide information about the medication, side effects, and how to take it. You can also help patients find the right medication for their condition.", # Provide instructions to the AI
    tools=[{"type": "file_search"}], # Define the tools that the AI can use
    model="gpt-4o", # Use the GPT-4 model and enable chat mode
    )

Uploads drug database to Vector Store

In [82]:
# Create a vector store called "Drug Database"
vector_store = client.beta.vector_stores.create(name="Drug Database")
 
# Ready the files for upload to OpenAI
file_paths = ["files/DrugsbyNDCClass3_090809.pdf", ]
file_streams = [open(path, "rb") for path in file_paths]
 
# Use the upload and poll SDK helper to upload the files, add them to the vector store,
# and poll the status of the file batch for completion.
file_batch = client.beta.vector_stores.file_batches.upload_and_poll(
  vector_store_id=vector_store.id, files=file_streams
)
 
# You can print the status and the file counts of the batch to see the result of this operation.
print(file_batch.status)
print(file_batch.file_counts)

completed
FileCounts(cancelled=0, completed=1, failed=0, in_progress=0, total=1)


Update assistant to use new Vector Store

In [83]:
assistant = client.beta.assistants.update(
  assistant_id=assistant.id,
  tool_resources={"file_search": {"vector_store_ids": [vector_store.id]}},
)

Initialize thread

In [80]:
thread = client.beta.threads.create() # Create a new conversation thread

Create message between assistant and user

In [81]:
messageInput = input("You: ") # Get the user's input

message = client.beta.threads.messages.create(
    thread_id=thread.id,
    role="user",
    content=messageInput
    )